# A very Naïve and Time Consuming Solution, we need to improve this with Trie
#### Edit 1: Stupid Naive approach
#### Edit 2: Peter Norvich's spell checker
#### Edit 3: Fixing Output

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
# Required Distance
!pip install textdistance

/kaggle/input/itmo-spelling-correction-autumn-2019/train.csv
/kaggle/input/itmo-spelling-correction-autumn-2019/no_fix.submission.csv
/kaggle/input/itmo-spelling-correction-autumn-2019/words.csv
/kaggle/input/itmo-spelling-correction-autumn-2019/evaluate.py


In [2]:
import textdistance
import time

In [108]:
train_data = pd.read_csv('/kaggle/input/itmo-spelling-correction-autumn-2019/train.csv')
words = pd.read_csv('/kaggle/input/itmo-spelling-correction-autumn-2019/words.csv').dropna().set_index('Id')['Freq'].T.to_dict()
test_data = pd.read_csv('/kaggle/input/itmo-spelling-correction-autumn-2019/no_fix.submission.csv').fillna('NaN', limit=1)

In [ ]:
test_data.fill('NaN2', limit=1)

In [109]:
test_data.iloc[258760: 258770]

,Id,Predicted
258760,HÓTPOT,HÓTPOT
258761,NaN,NaN
258762,СДОВА,СДОВА
258763,КУИНТОН,КУИНТОН
258764,КИНЗОЙ,КИНЗОЙ
258765,ТВОРЯТ,ТВОРЯТ
258766,VALLENSBÆK,VALLENSBÆK
258767,ЛОДЖИИ,ЛОДЖИИ
258768,ОСОЛДАТИТЬСЯ,ОСОЛДАТИТЬСЯ
258769,LIGHTSABER,LIGHTSABER


In [89]:
# Add nan to dictionary
words['nan'] = 0

In [8]:
train_data.head()

,Id,Expected
0,НЕЛОВКО,НЕЛОВКО
1,UNIZOOI,UNIZOO
2,ISIC,ISIC
3,DUNYADA,DUNYADA
4,ITALWAX,ITALWAX


In [9]:
# words

{'НЕЛОВКО': 55,
 'UNIZOOI': 20,
 'ISIC': 60,
 'DUNYADA': 131,
 'ITALWAX': 41,
 'ТАГЕРА': 173,
 'МДВ': 84,
 'ЄВРОПАРД': 16,
 'ҚҰҚЫН': 19,
 'ГЮЛЬХАНЕ': 30,
 'КПРТЕ': 29,
 'GREENALLS': 16,
 'СУЗГЕ': 24,
 'SHOTGUN': 236,
 'ВЫЕБЛ': 23,
 '456': 453,
 'KLOKKERHOLM': 32,
 'ПЕРЕТВОРЕННЯМ': 23,
 'GH1018': 82,
 'АНІМЕ': 989,
 'ВЫДАВАЕМЫЙ': 42,
 'РУСОФОБИЯ': 91,
 'НАЗЕМНО': 242,
 'ХИВЕ': 85,
 'ВЫКАЧИВАЕТ': 24,
 'TELEFUNKEN': 670,
 'FREEDOS': 31,
 'СТОЛБЧАТЫЕ': 41,
 'РАСКОВАННЫЕ': 37,
 'ПОЛИТОБОЗРЕНИЕ': 216,
 'WGN': 112,
 'УБЕЖДАЮЩЕГО': 20,
 'МОРЕПЛАВАНИЯ': 58,
 'HASMIK': 66,
 'BDR': 161,
 'TQDK': 151,
 'ТАВИЛ': 48,
 'БЭНКСИ': 474,
 'ՎԻՐՈՒՍՆԵՐ': 24,
 'НАБЛЮДАЙ': 18,
 'ХЕЛИДОНИУМ': 46,
 'НЕМЗЕР': 22,
 'GAYBBS': 44,
 'КВАРИАТИ': 43,
 'ШЕРКАЛ': 36,
 'СМОЛК': 18,
 'ЭУСТОМУ': 194,
 'ОТРЕПЬЕВ': 76,
 'ШУДРА': 26,
 'СОЛГИНСКОМ': 38,
 'ДЕМОСИСТЕМА': 480,
 'ИЗНАСИОВАНИЕ': 31,
 'КРАЙНЯ': 97,
 'ШАТЕР': 1996,
 'СИРЯЛ': 71,
 'ARCA': 216,
 'ГИДРОМЕТ': 153,
 'KUURAKETT': 76,
 'ОЛЬХОВКИ': 25,
 'БОКМАСИН': 21,
 'ПОД

In [10]:
print(len(test_data))
test_data.head()


362256


,Id,Predicted
0,SNAPDEAL,SNAPDEAL
1,ЧЕННЕЛИНГ,ЧЕННЕЛИНГ
2,ХАБИБЕ,ХАБИБЕ
3,ПОРНОСПЯЩИЕ,ПОРНОСПЯЩИЕ
4,ГЕРМЕТИЧНАЯ,ГЕРМЕТИЧНАЯ


In [12]:
# merged_data = test_data.merge(words, left_on='Predicted', right_on='Id').dropna()
# merged_data

#### We have all words in the words dataframe

In [13]:
distance_to_keep = [1, 2]
distance_function = textdistance.levenshtein


In [90]:
WORDS = words

def P(word, N=sum(WORDS.values())): 
    "Probability of `word`."
    return WORDS[word] / N

def correction(word):
    "Most probable spelling correction for word."
    return max(candidates(word), key=P)

def candidates(word): 
    "Generate possible spelling corrections for word."
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "The subset of `words` that appear in the dictionary of WORDS."
    if words:
        return set(w for w in words if w and w in WORDS)
    return set()

        

def edits1(word):
    "All edits that are one edit away from `word`."
    letters    = 'бвгджзклмнпрстфхцчшщаэыуояеёюийьъ'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "All edits that are two edits away from `word`."
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [91]:
correction('nan')

'nan'

In [92]:
start_time = time.time()
print(correction('ПОРНОСПЯЩИЕA'))
print("--- %s seconds ---" % (time.time() - start_time))

ПОРНОСПЯЩИЕ
--- 0.001432657241821289 seconds ---


In [93]:
start_time = time.time()
test_data['Predicted'] = test_data['Id'].apply(lambda x: correction(x))
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.4538202285766602 seconds ---


In [96]:
test_data.to_csv('submission.csv', index=False)